In [5]:
# nuclear/performance.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
              # nuclear/data/

# 1) Load operational reactors
df_op = pd.read_csv(r"C:\Users\Armstrong\nuclear\data\wn_all_countries_reactors.csv")
df_op_agg = (
    df_op
    .groupby("Country", as_index=False)["Capacity (MWe)"]
    .sum()
    .rename(columns={"Capacity (MWe)": "OperationalCapacity_MW"})
)
df_op_agg["Capacity_GW"] = df_op_agg["OperationalCapacity_MW"] / 1000

df_energy = pd.read_csv(r"C:\Users\Armstrong\nuclear\data\pris\trend_reports\energy availability.csv")
df_unit   = pd.read_csv(r"C:\Users\Armstrong\nuclear\data\pris\trend_reports\unit_capability.csv")
df_loss   = pd.read_csv(r"C:\Users\Armstrong\nuclear\data\pris\trend_reports\unplanned_capability_loss.csv")

import re

def latest_year_col(df):
    # Find columns whose name (as string) matches a 4-digit year
    year_cols = [
        c for c in df.columns
        if re.match(r'^\s*(\d{4})\s*$', str(c))
    ]
    if not year_cols:
        # Fallback: print them out for manual inspection
        print("Available columns:", df.columns.tolist())
        raise ValueError("No year-only columns found.")
    # Strip whitespace and convert to int
    years = [int(c.strip()) for c in year_cols]
    latest = max(years)
    # Return it *as string* to use in rename()
    return str(latest)

# Then rename:
year = latest_year_col(df_energy)
df_energy = df_energy.rename(columns={year: "EnergyAvail_pct"})

year = latest_year_col(df_unit)
df_unit   = df_unit.rename(columns={year: "UnitCap_pct"})

year = latest_year_col(df_loss)
df_loss   = df_loss.rename(columns={year: "Loss_pct"})

# 3) Merge performance metrics
df_perf = df_op_agg[["Country","Capacity_GW"]].copy()
for df_m, col in [(df_energy, "EnergyAvail_pct"),
                  (df_unit,   "UnitCap_pct"),
                  (df_loss,   "Loss_pct")]:
    df_perf = df_perf.merge(df_m[["Country",col]], on="Country", how="left")

# 4) Compute deltas vs. top-quartile
th_energy = df_perf["EnergyAvail_pct"].quantile(0.75)
th_unit   = df_perf["UnitCap_pct"].quantile(0.75)
th_loss   = df_perf["Loss_pct"].quantile(0.25)

df_perf["Delta_Energy"] = (th_energy - df_perf["EnergyAvail_pct"]).clip(lower=0)
df_perf["Delta_Unit"]   = (th_unit   - df_perf["UnitCap_pct"]).clip(lower=0)
df_perf["Delta_Loss"]   = (df_perf["Loss_pct"] - th_loss).clip(lower=0)

# 5) Dollar‐value estimate ($5M per %-point per GW)
df_perf["Value_Energy_M"] = df_perf["Delta_Energy"] * df_perf["Capacity_GW"] * 5
df_perf["Value_Unit_M"]   = df_perf["Delta_Unit"]   * df_perf["Capacity_GW"] * 5
df_perf["Value_Loss_M"]   = df_perf["Delta_Loss"]   * df_perf["Capacity_GW"] * 5

df_perf["TotalOpportunity_M"] = (
    df_perf["Value_Energy_M"] + 
    df_perf["Value_Unit_M"] + 
    df_perf["Value_Loss_M"]
)

# 6) Top 10
top10 = df_perf.nlargest(10, "TotalOpportunity_M").copy()

# --- Streamlit UI ---
st.title("Performance Benchmark & Service-Opportunity Map")

st.write("""
Estimates service‐opportunity based on lagging
energy availability, unit capability, and unplanned loss 
vs top‐quartile peers (assuming \$5 M per %-point per GW).
""")

st.dataframe(
    top10[[
        "Country","Capacity_GW",
        "EnergyAvail_pct","Delta_Energy","Value_Energy_M",
        "UnitCap_pct",  "Delta_Unit",  "Value_Unit_M",
        "Loss_pct",     "Delta_Loss",  "Value_Loss_M",
        "TotalOpportunity_M"
    ]].rename(columns={
        "Capacity_GW":           "Capacity (GW)",
        "EnergyAvail_pct":       "Energy Avail (%)",
        "Delta_Energy":          "ΔEnergy (%)",
        "Value_Energy_M":        "Energy $M",
        "UnitCap_pct":           "Unit Cap (%)",
        "Delta_Unit":            "ΔUnit (%)",
        "Value_Unit_M":          "Unit $M",
        "Loss_pct":              "Unplanned Loss (%)",
        "Delta_Loss":            "ΔLoss (%)",
        "Value_Loss_M":          "Loss $M",
        "TotalOpportunity_M":    "Total $M"
    }).set_index("Country")
)

fig, ax = plt.subplots(figsize=(9,5))
ax.bar(top10["Country"], top10["TotalOpportunity_M"])
ax.set_xticks(range(len(top10)))
ax.set_xticklabels(top10["Country"], rotation=45, ha="right")
ax.set_ylabel("Total Opportunity ($M)")
ax.set_title("Top 10 Service Opportunities by Country")
fig.tight_layout()

st.pyplot(fig)


Available columns: ['Year', 'Net Electrical Capacity, GW(e)', 'Number of Commercially Operated Reactors with Data', 'EAF [%]']


<>:84: SyntaxWarning: invalid escape sequence '\$'
<>:84: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22016\2890143188.py:84: SyntaxWarning: invalid escape sequence '\$'
  st.write("""
C:\Users\Armstrong\AppData\Local\Temp\ipykernel_22016\2890143188.py:84: SyntaxWarning: invalid escape sequence '\$'
  st.write("""


ValueError: No year-only columns found.

In [6]:
# nuclear/performance.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import re

# ————————————————
# 0) Paths (relative to this script)
BASE   = Path(__file__).parent          # nuclear/
DATA   = BASE / "data"                  # nuclear/data/
PRIS   = DATA / "pris"                  # nuclear/data/pris/
TRENDS = PRIS / "trend_reports"         # nuclear/data/pris/trend_reports/

# ————————————————
# 1) Load existing fleet capacity
df_op = pd.read_csv(DATA / "wn_all_countries_reactors.csv")
df_op_agg = (
    df_op
    .groupby("Country", as_index=False)["Capacity (MWe)"]
    .sum()
    .rename(columns={"Capacity (MWe)": "OperationalCapacity_MW"})
)
df_op_agg["Capacity_GW"] = df_op_agg["OperationalCapacity_MW"] / 1000

# ————————————————
# 2) Load “last-3-year” performance tables
#    (you should have these CSVs in trend_reports/)
df_e  = pd.read_csv(TRENDS / "Last3YearEnergyAvailability.csv")
df_u  = pd.read_csv(TRENDS / "Last3YearUnitCapability.csv")
df_l  = pd.read_csv(TRENDS / "Last3YearUnplannedCapacityLoss.csv")

def pick_latest_year_col(df):
    # find columns that are exactly a 4-digit year
    yrs = [c for c in df.columns if re.fullmatch(r"\s*\d{4}\s*", str(c))]
    if not yrs:
        raise ValueError(f"No year columns found in {df.columns.tolist()}")
    yrs = [int(c.strip()) for c in yrs]
    return str(max(yrs))

# Rename the latest-year column in each
col_e = pick_latest_year_col(df_e)
col_u = pick_latest_year_col(df_u)
col_l = pick_latest_year_col(df_l)

df_e = df_e.rename(columns={col_e: "EnergyAvail_pct"})
df_u = df_u.rename(columns={col_u: "UnitCap_pct"})
df_l = df_l.rename(columns={col_l: "Loss_pct"})

# Keep only Country + metric
df_e = df_e[["Country", "EnergyAvail_pct"]]
df_u = df_u[["Country", "UnitCap_pct"]]
df_l = df_l[["Country", "Loss_pct"]]

# ————————————————
# 3) Merge everything
df = df_op_agg[["Country", "Capacity_GW"]].copy()
for d, col in [(df_e, "EnergyAvail_pct"),
               (df_u, "UnitCap_pct"),
               (df_l, "Loss_pct")]:
    df = df.merge(d, on="Country", how="left")

# ————————————————
# 4) Compute top-quartile benchmarks
th_e = df["EnergyAvail_pct"].quantile(0.75)
th_u = df["UnitCap_pct"].quantile(0.75)
th_l = df["Loss_pct"].quantile(0.25)  # lower is better

# 5) Compute “deltas” (only positive shortfalls)
df["ΔEnergy"] = (th_e - df["EnergyAvail_pct"]).clip(lower=0)
df["ΔUnit"]   = (th_u - df["UnitCap_pct"]).clip(lower=0)
df["ΔLoss"]   = (df["Loss_pct"] - th_l).clip(lower=0)

# 6) Value = $5 M × Δ% × GW
df["Energy_$M"] = df["ΔEnergy"] * df["Capacity_GW"] * 5
df["Unit_$M"]   = df["ΔUnit"]   * df["Capacity_GW"] * 5
df["Loss_$M"]   = df["ΔLoss"]   * df["Capacity_GW"] * 5

df["Total_$M"] = df[["Energy_$M","Unit_$M","Loss_$M"]].sum(axis=1)

# 7) Top 10 countries by opportunity
top10 = df.nlargest(10, "Total_$M").reset_index(drop=True)

# ————————————————
# 8) Streamlit UI
st.title("Performance Benchmark & Service-Opportunity Map")

st.markdown(
    "This ranks the **Top 10 countries** where improving plant performance "
    "(Energy Availability, Unit Capability, Unplanned Loss) to top-quartile "
    "levels unlocks the largest revenue pools—assuming **$5 M per %-point** per GW."
)

st.dataframe(
    top10[[
        "Country", "Capacity_GW",
        "EnergyAvail_pct","ΔEnergy","Energy_$M",
        "UnitCap_pct",  "ΔUnit",  "Unit_$M",
        "Loss_pct",    "ΔLoss",   "Loss_$M",
        "Total_$M"
    ]].rename(columns={
        "Capacity_GW":       "Capacity (GW)",
        "EnergyAvail_pct":   "Energy Avail (%)",
        "ΔEnergy":           "ΔEnergy (%)",
        "Energy_$M":         "Energy $M",
        "UnitCap_pct":       "Unit Cap (%)",
        "ΔUnit":             "ΔUnit (%)",
        "Unit_$M":           "Unit $M",
        "Loss_pct":          "Unplanned Loss (%)",
        "ΔLoss":             "ΔLoss (%)",
        "Loss_$M":           "Loss $M",
        "Total_$M":          "Total $M"
    }).set_index("Country")
)

# 9) Matplotlib bar chart
fig, ax = plt.subplots(figsize=(9, 5))
ax.bar(top10["Country"], top10["Total_$M"], color="C2")
ax.set_xticks(range(len(top10)))
ax.set_xticklabels(top10["Country"], rotation=45, ha="right")
ax.set_ylabel("Total Opportunity ($M)")
ax.set_title("Top 10 Service Opportunities by Country")
fig.tight_layout()

st.pyplot(fig)


NameError: name '__file__' is not defined